In [1]:
%load_ext autoreload
%autoreload 2
# Set HuggingFace cache directory to scratch to save space.
import os
os.environ['HUGGINGFACE_HUB_CACHE'] = '/scratch/' + os.environ['USER'] + '/huggingface_cache'
CACHE_DIR = '/scratch/' + os.environ['USER'] + '/huggingface_cache'
# Optional; can help when memory is tight.
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

### Load HellaSwag Dataset

In [3]:
from datasets import load_dataset
dataset = load_dataset("Rowan/hellaswag", "en-US", split="train", cache_dir=CACHE_DIR)

import numpy as np


selected_indices = np.random.choice(len(dataset["ctx"]), 5, replace=False)
selected_prompts = [dataset["ctx"][i] for i in selected_indices]

selected_prompts

['A text intro leads into a picture of a dog and the same dog running along the yard. the dog',
 '[header] How to overcome fear of disease [title] Work with a therapist. [step] Therapy is generally considered one of the most effective ways to manage anxiety disorders, and illness anxiety disorder is no different. There are many different approaches to therapy.',
 '[header] How to respond to passive aggressive comments [title] Avoid reacting defensively. [step] When someone makes a passive aggressive comment, you may feel the need to defend yourself, or make accusations about them. Getting upset will likely do little good to change their habits.',
 '[header] How to select hearing protection [title] Familiarize yourself with noise reduction ratings (nrr). [step] Nrr is the standard rating system across all hearing protection devices. Under nrr, hearing devices are classified by their potential to limit decibels (db) in professional and occupational environments.',
 'He uses a tool to hit

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from IPython.display import display, HTML, Markdown

def generate(model, tokenizer, prompt, do_display=True, max_new_tokens=50):
    input_string = prompt
    inputs = tokenizer(input_string, return_tensors="pt", return_attention_mask=True).to('cuda')
    
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0)
    text = tokenizer.batch_decode(outputs)[0][len(input_string):]

    if do_display:
        display(HTML(f"<pre>{input_string}</pre><pre style='background-color: rgb(200, 255, 200, 1.0)'>{text}<pre>"))

    return inputs['input_ids'][0], outputs[0], text

def run_coding_sample(model, tokenizer, display=True):
    prompt = '''def print_prime(n):
       """
       Print all primes between 1 and n
       """'''
    return generate(model, tokenizer, prompt, display)

def run_coding_sample_2(model, tokenizer):
    prompt = '''def array_sum(array, nrows, ncols):
       """
       Use two for loops to add elements of an array.
       """'''
    return generate(model, tokenizer, prompt)

In [18]:
from hooked_phi import attach_hooks, detach_hooks

all_results = []
all_results_tokenized = []

# Returns a hook that can be used to ablate a set of neurons.
def ablate_neurons(mask):
    assert mask.shape[0] == model.config.num_hidden_layers
    assert mask.shape[1] == model.config.intermediate_size

    def hook(neurons, layer_idx):
        neurons[..., ~mask[layer_idx]] = 0
        return neurons

    return hook

# Ablate the last layer MLP.
for (start_layer, end_layer) in [(25, 26), (28, 29)]:
    if num_ablation_layers == 0:
        display(Markdown(f'## Baseline'))
    else:
        display(Markdown(f'## Ablating MLPs {32 - num_ablation_layers + 1} to {end_layer}'))
    
    mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
    if num_ablation_layers > 0:
        mask[start_layer:end_layer, :] = False
    
    attach_hooks(model.model, ablate_neurons(mask))
    
    # input_ids, input_and_completion_ids, result = run_coding_sample(model, tokenizer)
    input_ids, input_and_completion_ids, result = generate(model, tokenizer, "Instruct: Who was the first United States president?\n\nOutput: Their name was", do_display=True)
    if num_ablation_layers == 0:
        ground_truth = result
    
    all_results.append(result)

detach_hooks(model.model)

## Ablating MLPs 29 to 26

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Ablating MLPs 29 to 29

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [7]:
from evaluator import perplexity_evaluator

perplexity_result = perplexity_evaluator(preds=all_results)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 1/1 [00:00<00:00,  9.44it/s]


# Evaluate with HellaSwag

In [11]:
from evaluator import charcut_evaluator

ground_truths = []
predictions = {}

for prompt in selected_prompts:
    
    # Ablate the last layer MLP.
    for num_ablation_layers in range(0, 5, 2):
        
        mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
        if num_ablation_layers > 0:
            mask[-num_ablation_layers:, :] = False
        
        attach_hooks(model.model, ablate_neurons(mask))
        
        input_ids, input_and_completion_ids, result = generate(model, tokenizer, prompt, do_display=False)
        
        if num_ablation_layers == 0:
            ground_truths.append(prompt+result)
        
        if num_ablation_layers not in predictions:
            predictions[num_ablation_layers] = []
        
        predictions[num_ablation_layers].append(prompt+result)
        

    detach_hooks(model.model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

['A text intro leads into a picture of a dog and the same dog running along the yard. the dog is running with a ball in its mouth. the dog is running towards a person. the person is smiling and waving at the dog. the text says "Meet Max, the best dog in the world. He loves to play fetch and make new friends', '[header] How to overcome fear of disease [title] Work with a therapist. [step] Therapy is generally considered one of the most effective ways to manage anxiety disorders, and illness anxiety disorder is no different. There are many different approaches to therapy. [substeps] Cognitive behavioral therapy (CBT) is a common approach that helps people identify and change negative thought patterns. Exposure therapy is another approach that involves gradually exposing people to their fears in a safe and controlled environment. [substeps] Med', '[header] How to respond to passive aggressive comments [title] Avoid reacting defensively. [step] When someone makes a passive aggressive comme

In [15]:
print(ground_truths)

['A text intro leads into a picture of a dog and the same dog running along the yard. the dog is running with a ball in its mouth. the dog is running towards a person. the person is smiling and waving at the dog. the text says "Meet Max, the best dog in the world. He loves to play fetch and make new friends', '[header] How to overcome fear of disease [title] Work with a therapist. [step] Therapy is generally considered one of the most effective ways to manage anxiety disorders, and illness anxiety disorder is no different. There are many different approaches to therapy. [substeps] Cognitive behavioral therapy (CBT) is a common approach that helps people identify and change negative thought patterns. Exposure therapy is another approach that involves gradually exposing people to their fears in a safe and controlled environment. [substeps] Med', '[header] How to respond to passive aggressive comments [title] Avoid reacting defensively. [step] When someone makes a passive aggressive comme

In [17]:
for key, value in predictions.items():
    print(key)

0
2
4


In [22]:
charcut_evaluator()

False

In [37]:
# This is called "neuron vis", but can really be used for anything involving a score assigned to each token.
# In this case, visualize logprob
from neuron_visualization import basic_neuron_vis, basic_neuron_vis_signed

# Get granular nll estimates
display(Markdown(f'# Visualizing How Log-Likelihood Evolves, and For Which Tokens'))

# 1. Get ground truth.
detach_hooks(model.model)
input_ids, input_and_completion_ids, ground_truth = generate(model, tokenizer, "5 times 8 equals", do_display=False, max_new_tokens=3)
completion_ids = input_and_completion_ids[len(input_ids):]

# Calculate baseline logprobs.
predictions_for_next_token = model(input_ids=input_and_completion_ids.unsqueeze(0))
logits = predictions_for_next_token[0][0]
logits_for_output_tokens = logits[len(input_ids) - 1:-1]
logprobs_for_output_tokens = torch.log_softmax(logits_for_output_tokens, dim=-1)
baseline_logprobs_for_sampled_output_tokens = logprobs_for_output_tokens[
    torch.arange(logprobs_for_output_tokens.shape[0]),
    completion_ids
]

# 2. Ablate model. See which suddens suddenly become highly unlikely (by visualizing negative logprobs).
# for ablation_layer in range(31, 31 - 8 - 1, -1):
# # for ablation_layer in range(31, -1, -1):
#     mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
    
#     display(Markdown(f'## Ablating MLP {ablation_layer + 1} alone'))
#     mask[ablation_layer, :] = False

# for num_ablation_layers in range(0, 5, 1):
#     mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
#     if num_ablation_layers == 0:
#         display(Markdown(f'## Baseline'))
#     else:
#         display(Markdown(f'## Ablating MLPs {32 - num_ablation_layers + 1} to 32'))
for ablate_layer in [-1, 23, 24, 25, 26, 27, 28, 29, 30, 31]:
    # if num_ablation_layers > 0:
    #     mask[-num_ablation_layers:, :] = False
    if ablate_layer == -1:
        display(Markdown(f'## Baseline'))
    else:
        display(Markdown(f'## Ablating MLP {ablate_layer}'))

    mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)    
    if ablate_layer > 0:
        mask[ablate_layer, :] = False

    attach_hooks(model.model, ablate_neurons(mask))
    predictions_for_next_token = model(input_ids=input_and_completion_ids.unsqueeze(0))
    logits = predictions_for_next_token[0][0]
    logits_for_output_tokens = logits[len(input_ids) - 1:-1]
    logprobs_for_output_tokens = torch.log_softmax(logits_for_output_tokens, dim=-1)
    logprobs_for_sampled_output_tokens = logprobs_for_output_tokens[
        torch.arange(logprobs_for_output_tokens.shape[0]),
        completion_ids
    ]

    # visualized_tokens = outputs[0].cpu()[len(inputs['input_ids'][0].cpu()):]
    visualized_tokens = input_and_completion_ids.cpu()
    token_names = [
        tokenizer.decode(torch.tensor([visualized_tokens[i]]))
        for i in range(len(visualized_tokens))
    ]

    deflections = logprobs_for_sampled_output_tokens - baseline_logprobs_for_sampled_output_tokens

    colors = torch.cat([torch.zeros_like(input_ids), deflections])

    print("Average deflection:", deflections.mean().item())
    print("Worst deflection:", deflections.min().item())
    if ablate_layer != -1:
        print("Expected worst deflection (SHAP):", average_perturbations[ablate_layer])
    html = basic_neuron_vis_signed(token_names, colors, 1)
    display(HTML(html))

detach_hooks(model.model)


# Visualizing How Log-Likelihood Evolves, and For Which Tokens

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Baseline

Average deflection: 0.0
Worst deflection: 0.0


## Ablating MLP 23

...

Average deflection: -0.18967251479625702
Worst deflection: -0.2672095000743866
Expected worst deflection (SHAP): -0.2871393635869026


## Ablating MLP 30

Average deflection: -0.09360592812299728
Worst deflection: -0.2958889901638031
Expected worst deflection (SHAP): -0.25434938818216324


## Ablating MLP 31

Average deflection: -0.23373013734817505
Worst deflection: -0.850031852722168
Expected worst deflection (SHAP): -0.8379189703199599


# Using SHAP (SHapley Additive exPlanations)

In [36]:
# This is called "neuron vis", but can really be used for anything involving a score assigned to each token.
# In this case, visualize logprob
from neuron_visualization import basic_neuron_vis, basic_neuron_vis_signed
import tqdm

# Get granular nll estimates
display(Markdown(f'# Calculating Monte Carlo SHAP Scores Per Layer'))

# 1. Get ground truth.
detach_hooks(model.model)
input_ids, input_and_completion_ids, ground_truth = generate(model, tokenizer, "5 times 8 equals", do_display=False, max_new_tokens=3)
completion_ids = input_and_completion_ids[len(input_ids):]

print(tokenizer.decode(input_and_completion_ids))

# Calculate baseline logprobs.
predictions_for_next_token = model(input_ids=input_and_completion_ids.unsqueeze(0))
logits = predictions_for_next_token[0][0]
logits_for_output_tokens = logits[len(input_ids) - 1:-1]
logprobs_for_output_tokens = torch.log_softmax(logits_for_output_tokens, dim=-1)
baseline_logprobs_for_sampled_output_tokens = logprobs_for_output_tokens[
    torch.arange(logprobs_for_output_tokens.shape[0]),
    completion_ids
]

# 2. Ablate model. See which suddens suddenly become highly unlikely (by visualizing negative logprobs).
logged_perturbations = {}

num_coalitions = 128
for sampling_iteration in tqdm.tqdm(range(num_coalitions), desc='Running Monte Carlo sampling'):
    # Choose coalition.
    # coalition_size = torch.randint(3, 5, ())
    coalition_size = torch.randint(1, 3, ())
    # coalition_size = 1
    coalition = torch.randperm(32)[:coalition_size]

    # For each layer in this coalition, simulate its removal.
    mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
    if num_ablation_layers > 0:
        mask[coalition, :] = False

    attach_hooks(model.model, ablate_neurons(mask))
    predictions_for_next_token = model(input_ids=input_and_completion_ids.unsqueeze(0))
    logits = predictions_for_next_token[0][0]
    logits_for_output_tokens = logits[len(input_ids) - 1:-1]
    logprobs_for_output_tokens = torch.log_softmax(logits_for_output_tokens, dim=-1)
    logprobs_for_sampled_output_tokens_base = logprobs_for_output_tokens[
        torch.arange(logprobs_for_output_tokens.shape[0]),
        completion_ids
    ]
    detach_hooks(model.model)

    removed_features = []
    deflections = []
    
    for test_feature_index in range(len(coalition)):
        mask = torch.ones((model.config.num_hidden_layers, model.config.intermediate_size), dtype=torch.bool)
        new_coalition = torch.cat([coalition[:test_feature_index], coalition[test_feature_index + 1:]], dim=0)
        if num_ablation_layers > 0:
            mask[new_coalition, :] = False

        attach_hooks(model.model, ablate_neurons(mask))
        predictions_for_next_token = model(input_ids=input_and_completion_ids.unsqueeze(0))
        logits = predictions_for_next_token[0][0]
        logits_for_output_tokens = logits[len(input_ids) - 1:-1]
        logprobs_for_output_tokens = torch.log_softmax(logits_for_output_tokens, dim=-1)
        logprobs_for_sampled_output_tokens_probed = logprobs_for_output_tokens[
            torch.arange(logprobs_for_output_tokens.shape[0]),
            completion_ids
        ]
        detach_hooks(model.model)
        removed_features.append(test_feature_index)
        
        # "probed" has one less feature. So, the marginal contribution of the feature is measured by subtracting
        # base - probed.
        deflections.append(logprobs_for_sampled_output_tokens_base - logprobs_for_sampled_output_tokens_probed)

        # Log the perturbation
        feature_id = int(coalition[test_feature_index])
        if feature_id not in logged_perturbations:
            logged_perturbations[feature_id] = []

        perturbation = (logprobs_for_sampled_output_tokens_base - logprobs_for_sampled_output_tokens_probed).min()
        logged_perturbations[feature_id].append(perturbation.item())

# Calculate approximate SHAP scores based on the perturbations we've logged.
average_perturbations = {
    key: sum(values) / len(values)
    for (key, values) in logged_perturbations.items()
}

sorted_keys = sorted(average_perturbations.keys())

for key in sorted_keys:
    print(f"Layer {key}: {average_perturbations[key]:.3f}")


# Calculating Monte Carlo SHAP Scores Per Layer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5 times 8 equals 40.



Running Monte Carlo sampling: 100%|██████████| 128/128 [00:11<00:00, 11.19it/s]

Layer 0: -5.629
Layer 1: 0.070
Layer 2: 0.027
Layer 3: -0.722
Layer 4: -0.313
Layer 5: -0.050
Layer 6: -0.065
Layer 7: -0.098
Layer 8: -0.225
Layer 9: -0.108
Layer 10: -0.090
Layer 11: 0.012
Layer 12: -0.134
Layer 13: -0.069
Layer 14: -0.081
Layer 15: -0.280
Layer 16: 0.016
Layer 17: -0.009
Layer 18: -0.121
Layer 19: -0.062
Layer 20: -0.128
Layer 21: 0.005
Layer 22: -0.292
Layer 23: -0.557
Layer 24: -0.164
Layer 25: -0.118
Layer 26: -0.389
Layer 27: -0.222
Layer 28: -0.168
Layer 29: -0.287
Layer 30: -0.254
Layer 31: -0.838


# Viewing "kernel" of MLP intermediate states

If there MLP intermediate states represent recall coefficients - with the MLP encoder representing detection, and the MLP decoder representing triggering of the new memory - we would expect that most MLP intermediate states would be 0 (as only a sparse number of "memories" should be activated at any given time), or that MLP intermediate states would be correlated with meaningful concepts. Let's test this hypothesis.

In [ ]:
def log_neurons(neurons, layer_idx):
    if layer_idx == 0:
        log.append([])

    log[-1].append(neurons)
    
    return neurons

attach_hooks(model.model, log_neurons)

In [ ]:
log = []
inputs, outputs, text = run_coding_sample_2(model, tokenizer)

layer_id = 0

all_mlp_activations = []
for i in range(len(log)):
    # log[forward pass index][layer index][batch index, token index in forward pass, mlp neuron index]
    mlp_activations = log[i][layer_id][0, -1, :]
    all_mlp_activations.append(mlp_activations)

stacked = torch.stack(all_mlp_activations, dim=0)

# Scale each MLP intermediate neuron by the maximum absolute value
scale_by_dim = torch.max(stacked.abs(), dim=0, keepdims=True).values
stacked_scaled = stacked / (scale_by_dim + 1e-5)


In [ ]:
import matplotlib.pyplot as plt

plt.title("MLP activations: Last Layer (Unscaled)")
plt.hist(stacked.view(-1).cpu(), bins=25)
plt.xlabel("MLP activation level")
plt.yscale("log")
plt.ylabel("Frequency")
plt.show()

plt.title("MLP activations: Last Layer (Scaled per dim.)")
plt.hist(stacked_scaled.view(-1).cpu(), bins=25)
plt.xlabel("MLP activation level")
plt.yscale("log")
plt.ylabel("Frequency")
plt.show()

## Interpretation

It seems that MLP activations are very sparse. Therefore, it will hopefully be relatively simple to find meaningful MLP neurons to visualize.

To automatically determine which MLP activations are most interesting, I will calculate the variance of the activations.


In [ ]:
from neuron_visualization import basic_neuron_vis

variance = stacked.var(dim=0)
highest_to_lowest_variance = variance.argsort(descending=True)
visualized_tokens = outputs[0].cpu()[len(inputs['input_ids'][0].cpu()):]

token_names = [
    tokenizer.decode(torch.tensor([visualized_tokens[i]]))
    for i in range(len(visualized_tokens))
]

visualized_neurons = highest_to_lowest_variance[:100]

for neuron_i in range(len(visualized_neurons)):
    neuron_id = visualized_neurons[neuron_i]
    html = basic_neuron_vis(token_names, stacked[:, neuron_id], layer=layer_id, neuron_index=neuron_id)
    display(HTML(html))
